# Running experiments in Azure Machine Learning

In this lab, you will learn to run experiments in Azure Machine Learning from Azure Databricks. This lab will cover following exercises:

- Exercise 1: Running an Azure ML experiment on Databricks
- Exercise 2: Reviewing experiment metrics in Azure ML Studio

To install the required libraries please follow the instructions in the lab guide.

**Required Libraries**: 
* `azureml-sdk[databricks]` via PyPI
* `sklearn-pandas==2.1.0` via PyPI
* `azureml-mlflow` via PyPI

Run the following cell to load common libraries.

In [0]:
import os
import numpy as np
import pandas as pd
import pickle
import sklearn
import joblib
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Experiment, Run
from azureml.core.model import Model

print('The azureml.core version is {}'.format(azureml.core.VERSION))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-1988255026187189>:14
     12 from sklearn.impute import SimpleImputer
     13 from sklearn.preprocessing import StandardScaler, OneHotEncoder
---> 14 from sklearn_pandas import DataFrameMapper
     15 from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
     17 import matplotlib

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thre

## Connect to the AML workspace

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, and `workspace_name` as directed by the comments. Please note, you can copy the `subscription ID` and `resource group` name from the **Overview** page on the blade for the Azure ML workspace in the Azure portal.

In [0]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "XXX-XXXX-XXXX-XXXX-XXXX"

#Replace the name below with the name of your resource group
resource_group = "XXX"

#Replace the name below with the name of your Azure Machine Learning workspace
workspace_name = "aml-ws"

print("subscription_id:", subscription_id)
print("resource_group:", resource_group)
print("workspace_name:", workspace_name)

**Important Note**: You will be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

*Also note that the sign-on link and code only appear the first time in a session. If an authenticated session is already established, you won't be prompted to enter the code and authenticate when creating an instance of the Workspace.*

In [0]:
ws = Workspace(subscription_id, resource_group, workspace_name)
print(ws)
print('Workspace region:', ws.location)
print('Workspace configuration succeeded')

## Load the training data

In this notebook, we will be using a subset of NYC Taxi & Limousine Commission - green taxi trip records available from [Azure Open Datasets]( https://azure.microsoft.com/en-us/services/open-datasets/). The data is enriched with holiday and weather data. Each row of the table represents a taxi ride that includes columns such as number of passengers, trip distance, datetime information, holiday and weather information, and the taxi fare for the trip.

Run the following cell to load the table into a Spark dataframe and reivew the dataframe.

In [0]:
dataset = spark.sql("select * from nyc_taxi").toPandas()
display(dataset)

## Exercise 1: Running an Azure ML experiment on Databricks

### Use MLflow with Azure Machine Learning for Model Training

In the subsequent cells you will learn to do the following:
- Set up MLflow tracking URI so as to use Azure ML
- Create MLflow experiment – this will create a corresponding experiment in Azure ML Workspace
- Train a model on Azure Databricks cluster while logging metrics and artifacts using MLflow

After this notebook, you should return to the **lab guide** and follow instructions to review the model performance metrics and training artifacts in the Azure Machine Learning workspace.

#### Set MLflow tracking URI

Set the MLflow tracking URI to point to your Azure ML Workspace. The subsequent logging calls from MLflow APIs will go to Azure ML services and will be tracked under your Workspace.

In [0]:
import mlflow
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
print("MLflow tracking URI to point to your Azure ML Workspace setup complete.")

#### Configure experiment

In [0]:
experiment_name = 'MLflow-AML-Exercise'
mlflow.set_experiment(experiment_name)
print("Experiment setup complete.")

#### Train Model and Log Metrics and Artifacts

Now you are ready to train the model. Run the cell below to do the following:
-	Train model
-	Evaluate model
-	Log evaluation metrics
-   Log artifact: Evaluation graph
-   Save model
-   Log artifact: Trained model

Note that the metrics and artifacts will be saved in your `AML Experiment Run`.

In [0]:
print("Training model...")
output_folder = 'outputs'
model_file_name = 'nyc-taxi.pkl'
dbutils.fs.mkdirs(output_folder)
model_file_path = os.path.join('/dbfs', output_folder, model_file_name)

with mlflow.start_run() as run:
  df = dataset.dropna(subset=['totalAmount'])
  x_df = df.drop(['totalAmount'], axis=1)
  y_df = df['totalAmount']

  X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=0)

  numerical = ['passengerCount', 'tripDistance', 'snowDepth', 'precipTime', 'precipDepth', 'temperature']
  categorical = ['hour_of_day', 'day_of_week', 'month_num', 'normalizeHolidayName', 'isPaidTimeOff']

  numeric_transformations = [([f], Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])) for f in numerical]
    
  categorical_transformations = [([f], OneHotEncoder(handle_unknown='ignore', sparse=False)) for f in categorical]

  transformations = numeric_transformations + categorical_transformations

  clf = Pipeline(steps=[('preprocessor', DataFrameMapper(transformations, df_out=True)), 
                        ('regressor', GradientBoostingRegressor())])

  clf.fit(X_train, y_train)
  
  y_predict = clf.predict(X_test)
  y_actual = y_test.values.flatten().tolist()
  
  rmse = math.sqrt(mean_squared_error(y_actual, y_predict))
  mlflow.log_metric('rmse', rmse)
  mae = mean_absolute_error(y_actual, y_predict)
  mlflow.log_metric('mae', mae)
  r2 = r2_score(y_actual, y_predict)
  mlflow.log_metric('R2 score', r2)
  
  plt.figure(figsize=(10,10))
  plt.scatter(y_actual, y_predict, c='crimson')
  plt.yscale('log')
  plt.xscale('log')

  p1 = max(max(y_predict), max(y_actual))
  p2 = min(min(y_predict), min(y_actual))
  plt.plot([p1, p2], [p1, p2], 'b-')
  plt.xlabel('True Values', fontsize=15)
  plt.ylabel('Predictions', fontsize=15)
  plt.axis('equal')
  
  results_graph = os.path.join('/dbfs', output_folder, 'results.png')
  plt.savefig(results_graph)
  mlflow.log_artifact(results_graph)
  
  joblib.dump(clf, open(model_file_path,'wb'))
  mlflow.log_artifact(model_file_path)

#### View the Experiment Run in Azure Machine Learning Workspace

Run the cell below and then **right-click** on **Link to Azure Machine Learning studio** link below to open the `AML Experiment Run Details` page in a **new browser tab**.

In [0]:
list(ws.experiments[experiment_name].get_runs())[0]

## Exercise 2: Reviewing experiment metrics in Azure ML Studio

Return to the `lab guide` and follow instructions to review the model performance metrics and training artifacts in the Azure Machine Learning workspace.